**Train on whole dataset and test on test set**

In [1]:
import os
import pandas as pd 
import numpy as np

from expert_tree import get_expert_tree_results, Expert_Tree
#from wrapper import best_first_search_mg

from sklearn.metrics import confusion_matrix

In [11]:
test_set_dir = 'web_data_ML_correct.csv'
data_test = pd.read_csv(test_set_dir)
#data_test.head(5)
print(data_test.shape)

(334, 13)


In [12]:
data_test

,Username,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,ChestWallSwelling,Age,Lymphedema_Dx
0,8,1,1,3.0,2,2,3,2,10,3.0,0,8,1
1,9,2,3,2.0,1,1,3,2,13,9.0,2,1,0
2,10,1,1,0.0,0,3,1,3,15,5.5,0,9,1
3,11,3,2,0.0,1,1,1,2,13,5.5,1,8,1
4,12,0,3,0.0,0,0,3,2,7,4.0,0,6,1
5,13,4,4,3.0,0,4,4,4,19,12.0,3,1,1
6,14,4,3,0.0,3,4,2,3,14,10.0,3,12,1
7,15,2,2,3.0,2,2,2,2,14,2.0,3,5,1
8,16,1,3,2.0,0,2,2,2,19,2.0,1,1,1
9,17,1,3,0.0,0,4,0,4,8,29.0,0,11,1


In [13]:

# fill 0 for missing data
for column in ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'ChestWallSwelling']:
    data_test[column].fillna(0, inplace=True)
# drop data missing important feature
data_test = data_test.dropna(subset=['TIME_LAPSE'])#, 'LVC'])
# delete Username
#data_test = data_test.drop(columns='Username')
# convert string to float
#data_test = data_test.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data_test = data_test.astype(np.float64)
# drop rows containing nan
data_test = data_test.dropna()
# drop data within a 6 months
data_test = data_test[data_test.TIME_LAPSE >=0.5]
# add log of time elapsed
#data_test['TIME_LAPSE_LOG'] = np.log(data_test['TIME_LAPSE'])

print('shape:')
print(data_test.shape)

shape:
(307, 13)


In [8]:
#add low and high LVC values
#data_test['low_LVC'] = [0.01]*data_test.shape[0]
#data_test['middle_LVC'] = [0.04]*data_test.shape[0]
#data_test['high_LVC'] = [0.10]*data_test.shape[0]

In [14]:
#put on dummy values for fluid
data_test['fluid_total'] = [None]*data_test.shape[0]

In [16]:
#put on dummy values for fluid
data_test['LVC'] = [None]*data_test.shape[0]

In [17]:
data_test

,Username,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,ChestWallSwelling,Age,Lymphedema_Dx,fluid_total,LVC
0,8.0,1.0,1.0,3.0,2.0,2.0,3.0,2.0,10.0,3.0,0.0,8.0,1.0,None,None
1,9.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,13.0,9.0,2.0,1.0,0.0,None,None
2,10.0,1.0,1.0,0.0,0.0,3.0,1.0,3.0,15.0,5.5,0.0,9.0,1.0,None,None
3,11.0,3.0,2.0,0.0,1.0,1.0,1.0,2.0,13.0,5.5,1.0,8.0,1.0,None,None
4,12.0,0.0,3.0,0.0,0.0,0.0,3.0,2.0,7.0,4.0,0.0,6.0,1.0,None,None
5,13.0,4.0,4.0,3.0,0.0,4.0,4.0,4.0,19.0,12.0,3.0,1.0,1.0,None,None
6,14.0,4.0,3.0,0.0,3.0,4.0,2.0,3.0,14.0,10.0,3.0,12.0,1.0,None,None
7,15.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,14.0,2.0,3.0,5.0,1.0,None,None
8,16.0,1.0,3.0,2.0,0.0,2.0,2.0,2.0,19.0,2.0,1.0,1.0,1.0,None,None
9,17.0,1.0,3.0,0.0,0.0,4.0,0.0,4.0,8.0,29.0,0.0,11.0,1.0,None,None


In [22]:
def generate_labels_basedon_LVC(lvc_val, df):
    df.drop(columns=['LVC'])
    df['LVC'] = [lvc_val]*df.shape[0]
    expert_result, expert_labels, _ = get_expert_tree_results(df, False,3)
    print(df)
    return expert_result, expert_labels

In [26]:
def try_different_lvc():
    expert_result, expert_labels = generate_labels_basedon_LVC(0.01, data_test)
    data_test['low_LVC_expert_value'] = expert_result
    data_test['low_LVC_expert_label'] = expert_labels
    expert_result2, expert_labels2 = generate_labels_basedon_LVC(0.10, data_test)
    data_test['high_LVC_expert_value'] = expert_result2
    data_test['high_LVC_expert_label'] = expert_labels2
    

In [28]:
try_different_lvc()

     Username  Mobility  ArmSwelling  BreastSwelling  Skin  PAS  FHT  \
0         8.0       1.0          1.0             3.0   2.0  2.0  3.0   
1         9.0       2.0          3.0             2.0   1.0  1.0  3.0   
2        10.0       1.0          1.0             0.0   0.0  3.0  1.0   
3        11.0       3.0          2.0             0.0   1.0  1.0  1.0   
4        12.0       0.0          3.0             0.0   0.0  0.0  3.0   
5        13.0       4.0          4.0             3.0   0.0  4.0  4.0   
6        14.0       4.0          3.0             0.0   3.0  4.0  2.0   
7        15.0       2.0          2.0             3.0   2.0  2.0  2.0   
8        16.0       1.0          3.0             2.0   0.0  2.0  2.0   
9        17.0       1.0          3.0             0.0   0.0  4.0  0.0   
11       20.0       4.0          2.0             0.0   0.0  2.0  3.0   
12       22.0       3.0          3.0             0.0   0.0  0.0  3.0   
13       23.0       0.0          1.0             0.0   0.0  0.0 

     Username  Mobility  ArmSwelling  BreastSwelling  Skin  PAS  FHT  \
0         8.0       1.0          1.0             3.0   2.0  2.0  3.0   
1         9.0       2.0          3.0             2.0   1.0  1.0  3.0   
2        10.0       1.0          1.0             0.0   0.0  3.0  1.0   
3        11.0       3.0          2.0             0.0   1.0  1.0  1.0   
4        12.0       0.0          3.0             0.0   0.0  0.0  3.0   
5        13.0       4.0          4.0             3.0   0.0  4.0  4.0   
6        14.0       4.0          3.0             0.0   3.0  4.0  2.0   
7        15.0       2.0          2.0             3.0   2.0  2.0  2.0   
8        16.0       1.0          3.0             2.0   0.0  2.0  2.0   
9        17.0       1.0          3.0             0.0   0.0  4.0  0.0   
11       20.0       4.0          2.0             0.0   0.0  2.0  3.0   
12       22.0       3.0          3.0             0.0   0.0  0.0  3.0   
13       23.0       0.0          1.0             0.0   0.0  0.0 

In [29]:
data_test

,Username,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,ChestWallSwelling,Age,Lymphedema_Dx,fluid_total,LVC,low_LVC_expert_value,low_LVC_expert_label,high_LVC_expert_value,high_LVC_expert_label
0,8.0,1.0,1.0,3.0,2.0,2.0,3.0,2.0,10.0,3.0,0.0,8.0,1.0,None,0.1,2,Moderate,2,Severe
1,9.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,13.0,9.0,2.0,1.0,0.0,None,0.1,2,Moderate,2,Severe
2,10.0,1.0,1.0,0.0,0.0,3.0,1.0,3.0,15.0,5.5,0.0,9.0,1.0,None,0.1,2,Moderate,2,Severe
3,11.0,3.0,2.0,0.0,1.0,1.0,1.0,2.0,13.0,5.5,1.0,8.0,1.0,None,0.1,2,Moderate,2,Severe
4,12.0,0.0,3.0,0.0,0.0,0.0,3.0,2.0,7.0,4.0,0.0,6.0,1.0,None,0.1,1,Mild,2,Severe
5,13.0,4.0,4.0,3.0,0.0,4.0,4.0,4.0,19.0,12.0,3.0,1.0,1.0,None,0.1,2,Moderate,2,Severe
6,14.0,4.0,3.0,0.0,3.0,4.0,2.0,3.0,14.0,10.0,3.0,12.0,1.0,None,0.1,2,Moderate,2,Severe
7,15.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,14.0,2.0,3.0,5.0,1.0,None,0.1,2,Moderate,2,Severe
8,16.0,1.0,3.0,2.0,0.0,2.0,2.0,2.0,19.0,2.0,1.0,1.0,1.0,None,0.1,2,Moderate,2,Severe
9,17.0,1.0,3.0,0.0,0.0,4.0,0.0,4.0,8.0,29.0,0.0,11.0,1.0,None,0.1,1,Mild,2,Severe


In [39]:
clinical_labels = list(data_test['Lymphedema_Dx'])

In [43]:
low_lvc_expert_labels = np.array(data_test['low_LVC_expert_value'])
high_lvc_expert_labels = np.array(data_test['high_LVC_expert_value'])

In [48]:
#replace 2 with 1 to classify with
low_lvc_expert_labels[low_lvc_expert_labels==2]=1
high_lvc_expert_labels[high_lvc_expert_labels==2]=1

EVALUATE THE RESULTS

In [51]:
from sklearn.metrics import confusion_matrix


In [54]:
# when LVC is 0.01
CM_low = confusion_matrix(clinical_labels, low_lvc_expert_labels)

In [55]:
CM_low

array([[ 50,  87],
       [  6, 164]], dtype=int64)

In [56]:
#When LVC is 0.1
CM_high = confusion_matrix(clinical_labels, high_lvc_expert_labels)

In [57]:
CM_high

array([[ 51,  86],
       [  6, 164]], dtype=int64)